# Data Preparation

required # of contacts, when to pay, churnability

In [1]:
# import required libraries

import pandas as pd
import numpy as np

In [2]:
# read datasets

claims = pd.read_csv('./claim.csv')
actions = pd.read_csv('./Action.csv')
collections = pd.read_csv('./collection.csv')

### Merging *Collection* data and debt ages calculation

Starting by computing how many times each debt is contacted in each day:

In [3]:
actions['contact_date'] = pd.to_datetime(actions.contact_date, format='mixed', errors='coerce')
df = actions.groupby(['contact_date', 'debt_id']).size().reset_index(name='number_of_contact')
df.sample(5)

,contact_date,debt_id,number_of_contact
167995,2022-06-01,207246,1
255331,2022-11-28,263082,1
33085,2021-06-17,21755,1
81973,2021-10-24,41593,1
51438,2021-08-09,25022,1


Then, combining the `enter_debt` date to each unique debt with *actions* data to calculate `debt ages`:

In [4]:
collections[pd.to_datetime(collections.enter_date, errors='coerce').isna()]

,debt_id,collector_id,enter_date,amount_outstanding,amount,service_end_date,agent_id
91925,285708,-1,NaN,NaN,NaN,NaN,-1
95334,289117,-1,NaN,NaN,NaN,NaN,-1
95335,289118,-1,NaN,NaN,NaN,NaN,-1
95336,289119,-1,NaN,NaN,NaN,NaN,-1
95337,289120,-1,NaN,NaN,NaN,NaN,-1
...,...,...,...,...,...,...,...
126177,323533,-1,NaN,NaN,NaN,NaN,-1
126178,323534,-1,NaN,NaN,NaN,NaN,-1
126179,323535,-1,NaN,NaN,NaN,NaN,-1
126180,323536,-1,NaN,NaN,NaN,NaN,-1


All of these are **NaN** values. The -1 value here represnets also a **NaN** value. Since they have not any information, we can drop them:

In [5]:
collections.drop(collections[pd.to_datetime(collections.enter_date, errors='coerce').isna()].index, inplace=True)

Next, I combine the `enter_debt` date to each with *actions* data to calculate `debt ages`:

In [6]:
df['enter_date'] = df.debt_id.map(
    dict(
        zip(collections.debt_id, pd.to_datetime(collections.enter_date, errors='coerce'))
    )
)

But wait, there are some debts in *collection* data are not actioned at all:

In [7]:
np.setdiff1d(collections.debt_id.unique(), actions.debt_id.unique()).shape

(10266,)

In the same time, some `contact_date` are entered incorrectly (e.g. some of them are grater than thier `enter_date`).

Next, let's calculate the `debt ages` for valid debt dates:

In [8]:
valid_debt_maks = (df['contact_date'].dt.year.between(2020, 2024)) & (df['contact_date'] >= df['enter_date'])
df.loc[valid_debt_maks, 'debt_age'] = df[valid_debt_maks].contact_date - df[valid_debt_maks].enter_date

Now, let's find how many there are incorrectly entered date:

In [9]:
df[~valid_debt_maks].shape[0]

11934

11934 from about 500,000 are incorrectly entered.

For them I will assign the mean `debt age` from those which are entered correctly and have some correctly entered records based on the `agent_id` dealing with:

In [10]:
df['agent_id'] = df.debt_id.map(dict(zip(collections['debt_id'], collections['agent_id']))) # combine the `agent_id` from collection data

grouped_valid_df = df[valid_debt_maks].groupby(by=['debt_id', 'agent_id'])['debt_age'].mean().reset_index(name='mean_debt_age')
df.loc[~valid_debt_maks, 'debt_age'] = df.loc[~valid_debt_maks].debt_id.map(
                                                                             dict(zip(grouped_valid_df.debt_id, grouped_valid_df.mean_debt_age))
                                                                    )

But not all of them has some valid contact date records:

In [11]:
df.isna().sum()

contact_date            0
debt_id                 0
number_of_contact       0
enter_date             12
debt_age             2607
agent_id               12
dtype: int64

Well, for those with **nan** values we can impute the global mean for them or continue analysis to find the best imputed values. Imputing the global mean, since the number of wrongly entered contact date are relativly small to the total number of records, may be available choice (i.e. they will not skew the distribution). But I'll leave them for now.

Next, I assign the `collector_id`, `amount`, and `service_end_date` for each debt:

In [12]:
df['collector_id'] = df.debt_id.map(dict(zip(collections['debt_id'], collections['collector_id'])))
df['amount'] = df.debt_id.map(dict(zip(collections['debt_id'], collections['amount'])))
df['service_end_date'] = df.debt_id.map(dict(zip(collections['debt_id'], pd.to_datetime(collections.service_end_date, format='mixed', errors='coerce'))))
df.isna().sum()

contact_date             0
debt_id                  0
number_of_contact        0
enter_date              12
debt_age              2607
agent_id                12
collector_id            12
amount                  19
service_end_date     23046
dtype: int64

There are about 23 thousand missed `service_end_date` values. These missing values are get generated because the `service_end_date` column itself from the collections data contains about 3000 debt with out `service_end_date` values.

Backing to the `debt_age`, on other valid solution is to use a KNN imputer model. However, I will impute the global median of `debt ages` for each `agent_id`:

In [13]:
average_debt_age_for_agent = df.groupby(by=['agent_id'])['debt_age'].median().reset_index(name='average_debt_age_for_agent')
df.loc[df.debt_age.isna(), 'debt_age'] = df[df.debt_age.isna()].agent_id.map(dict(zip(average_debt_age_for_agent.agent_id, average_debt_age_for_agent.average_debt_age_for_agent)))

Now, let's check for **nan** values:

In [14]:
df[df.debt_age.isna()]

,contact_date,debt_id,number_of_contact,enter_date,debt_age,agent_id,collector_id,amount,service_end_date
2,2020-02-25,4691,1,2020-12-01,NaT,32.0,25.0,30000.0,NaT
4,2020-08-19,83,3,NaT,NaT,NaN,NaN,NaN,NaT
339,2020-09-14,83,2,NaT,NaT,NaN,NaN,NaN,NaT
530,2020-09-21,83,1,NaT,NaT,NaN,NaN,NaN,NaT
162071,2022-05-10,244126,1,NaT,NaT,NaN,NaN,NaN,NaT
164017,2022-05-17,244052,1,NaT,NaT,NaN,NaN,NaN,NaT
164018,2022-05-17,244174,1,NaT,NaT,NaN,NaN,NaN,NaT
166949,2022-05-29,244052,1,NaT,NaT,NaN,NaN,NaN,NaT
198363,2022-08-14,244074,1,NaT,NaT,NaN,NaN,NaN,NaT
198364,2022-08-14,244083,1,NaT,NaT,NaN,NaN,NaN,NaT


Well, the rest **nans** here do not carry any information for us. In addition, they have no any appear in the *claims* data so we can drop them:

In [15]:
_ = df[(df.debt_age.isna())].debt_id
claims[claims.debt_id.isin(_)]

,debt_id,collected_amount,date_of_trans


As, you can see from the previous cell, so let's drop them:

In [16]:
df.drop(_.index, inplace=True)

In [17]:
df.sample(5)

,contact_date,debt_id,number_of_contact,enter_date,debt_age,agent_id,collector_id,amount,service_end_date
139835,2022-02-27,210326,1,2022-01-22,36 days,14.0,16.0,439.9250,2021-10-06
410944,2024-02-11,325899,1,2024-01-31,11 days,3.0,19.0,479.0800,2023-11-29
268696,2022-12-17,213684,1,2022-02-17,303 days,14.0,21.0,19.8720,2013-01-27
233011,2022-10-18,212641,1,2022-02-17,243 days,14.0,14.0,13.2920,2013-01-27
351922,2023-08-29,294932,1,2023-07-18,42 days,2093.0,10.0,152.7777,2020-06-07


Last column in the *collection* data is `amount_outstanding` I will leave it for now.

### Merging *Claims* data
Next, we have the *claims* data:

In [18]:
claims.sample(5)

,debt_id,collected_amount,date_of_trans
2401,20750,10.5056,2021-06-15
27091,259577,21.0540,2023-05-31
23176,251842,60.0000,2022-11-30
6301,19974,74.8400,2021-08-31
21653,246281,20.0000,2022-09-05


Starting by checking for debts which are not in the new dataset *df*:

In [19]:
np.setdiff1d(claims.debt_id.unique(), df.debt_id.unique()).shape

(239,)

A little bad number. I'll drop them since we will not leverage from them in the context of the new dataset *df* (i.e. they have not any information for other columns):

In [20]:
_ = np.setdiff1d(claims.debt_id.unique(), df.debt_id.unique())
claims.drop(claims[claims.debt_id.isin(_)].index, inplace=True)

Now, let's find how much each contact contribute to the collection process:

In [21]:
claims['date_of_trans'] = pd.to_datetime(claims.date_of_trans, format='mixed', errors='coerce')

In [22]:
df['transferred_amount'] = 0
df['date_of_trans'] = df['contact_date'].copy() # assuming each transformation is done on the same contact date just for now
                                                # just imagine it as an initial value before merging the `date_of_trans` column.
    
df_unasigned = pd.DataFrame(columns=claims.columns)

In the next cell, I assign the `transferred amount` to the nearest `contact_date` happen before or at `the date of the transformation`. It may takes a few minutes to run:

In [23]:
debts_id = claims.debt_id.unique()
for debt_id in debts_id:
    a = claims[claims['debt_id'] == debt_id]
    b = df[df['debt_id'] == debt_id]
    for trans in a.index:
        index = np.searchsorted(b['contact_date'], a.loc[[trans], 'date_of_trans'])[0]
        if index >= len(b) or b.iloc[index].contact_date > a.loc[trans, 'date_of_trans']:
            index -= 1
        if index >= 0:
            df.loc[b.iloc[[index]].index[0], 'transferred_amount'] += a.loc[trans, 'collected_amount']
            df.loc[b.iloc[[index]].index[0], 'date_of_trans'] = a.loc[trans, 'date_of_trans']
        else:
            df_unasigned = pd.concat([df_unasigned, a.loc[[trans]]])

df.sample(10)

,contact_date,debt_id,number_of_contact,enter_date,debt_age,agent_id,collector_id,amount,service_end_date,transferred_amount,date_of_trans
414140,2024-02-25,325845,1,2024-01-31,25 days 00:00:00,3.0,19.0,664.8500,2023-11-30,0.0,2024-02-25
414818,2024-02-28,245040,1,2022-05-23,646 days 00:00:00,14.0,21.0,396.1623,2021-12-22,0.0,2024-02-28
353314,2023-09-02,296664,1,2023-07-22,42 days 00:00:00,14.0,16.0,125.8634,2023-02-21,0.0,2023-09-02
187994,2022-07-21,214792,1,2022-02-17,154 days 00:00:00,14.0,16.0,10.4320,2013-04-29,0.0,2022-07-21
446819,2026-10-27,313511,1,2023-09-03,43 days 08:00:00,14.0,18.0,121.6107,2023-04-05,0.0,2026-10-27
233973,2022-10-18,256984,1,2022-09-25,23 days 00:00:00,14.0,16.0,545.4954,2022-04-24,0.0,2022-10-18
300897,2023-03-06,285386,2,2023-02-20,14 days 00:00:00,14.0,16.0,64.7703,2022-09-07,0.0,2023-03-06
146019,2022-03-15,201549,1,2021-11-12,123 days 00:00:00,14.0,28.0,364.1090,2021-10-23,0.0,2022-03-15
2989,2021-02-03,6157,1,2021-01-07,27 days 00:00:00,14.0,14.0,399.9031,2020-11-23,0.0,2021-02-03
275278,2022-12-27,225518,1,2022-02-17,313 days 00:00:00,14.0,16.0,92.6930,2017-11-06,0.0,2022-12-27


However, there are still some transactions but they were done without a prior contact at all. I'll discard them because, I mean was the transfer done within the company? Did the debtors pay through the company, or directly through the agency? (*ask follow up question on how to deal with them*):

In [24]:
(f'Number of transactions with out pre contact {df_unasigned.shape[0]}', f'Total amount {round(df_unasigned.collected_amount.sum(), 2)}'), (f'Number of transactions with a pre contact {(df.transferred_amount != 0).sum()}', f'Total amount {round(df.transferred_amount.sum(), 2)}')

(('Number of transactions with out pre contact 4375',
  'Total amount 138006.99'),
 ('Number of transactions with a pre contact 26678',
  'Total amount 1151985.09'))

Next, I merege the `outstanding amount` column from *collection* data. Starting by checking for **null** values:

In [25]:
outstandings = collections[['debt_id', 'amount_outstanding']]
outstandings.isna().sum()

debt_id                  0
amount_outstanding    4304
dtype: int64

Well, we can find the `outstanding` for those which are in *claims* data:

In [26]:
findable_outstanding = claims[claims.debt_id.isin(outstandings[outstandings.amount_outstanding.isna()].debt_id.unique())]
findable_outstanding = findable_outstanding.groupby('debt_id')['collected_amount'].sum().reset_index(name='total_transferred_amount')
findable_outstanding

,debt_id,total_transferred_amount
0,17240,10.50
1,19818,0.74
2,19844,0.95
3,19922,30.00
4,19928,76.54
5,19977,124.31


Let's compare them with their `amounts`:

In [27]:
findable_outstanding_amount = pd.concat(
    [
        findable_outstanding, 
        df[df.debt_id.isin(findable_outstanding.debt_id.unique())].groupby('debt_id').first().reset_index().drop('debt_id', axis=1)
    ], 
    axis=1)[['debt_id', 'total_transferred_amount', 'amount']]
findable_outstanding_amount

,debt_id,total_transferred_amount,amount
0,17240,10.50,60.51
1,19818,0.74,5.17
2,19844,0.95,0.56
3,19922,30.00,43.22
4,19928,76.54,76.54
5,19977,124.31,62.29


Since the `amounts` are relatively small, we can consider that the `outstanding amount` is the same as the `amount`.

In [28]:
collections.loc[collections.debt_id.isin(findable_outstanding_amount.debt_id), 'amount_outstanding'] = findable_outstanding_amount.amount.to_numpy()
collections[collections.debt_id.isin(findable_outstanding_amount.debt_id)]

,debt_id,collector_id,enter_date,amount_outstanding,amount,service_end_date,agent_id
11751,17240,19,2021-05-25,60.51,60.51,2015-11-29,1
14329,19818,16,2021-05-25,5.17,5.17,2019-03-21,1
14355,19844,30,2021-05-25,0.56,0.56,2021-01-31,1
14433,19922,30,2021-05-25,43.22,43.22,2021-04-29,1
14439,19928,30,2021-05-25,76.54,76.54,2021-04-29,1
14488,19977,21,2021-05-25,62.29,62.29,2021-04-29,1


Well, for the rest of nulls `amount_outstanding` I'll also impute the value of thier `amounts` for them:

In [29]:
collections.loc[collections.amount_outstanding.isna(), 'amount_outstanding'] = collections.loc[collections.amount_outstanding.isna(), 'amount']
collections[['debt_id', 'amount_outstanding']].isna().sum()

debt_id                0
amount_outstanding    16
dtype: int64

Great! Now continuing the merging process:

In [30]:
df['amount_outstanding'] = df.debt_id.map(
    collections.set_index('debt_id')['amount_outstanding'].to_dict()
)

Well, to compute the right `outstanding_amount` after each transformation, we should subtract the `transferred_amount` in each contact. let's prepare the dataframe first:

In [31]:
df.sort_values(by=['debt_id', 'contact_date'], inplace=True)
df.head()

,contact_date,debt_id,number_of_contact,enter_date,debt_age,agent_id,collector_id,amount,service_end_date,transferred_amount,date_of_trans,amount_outstanding
5,2020-08-19,84,2,2020-08-18,1 days,2.0,21.0,43637.0,NaT,0.0,2020-08-19,18666.0
117,2020-09-07,84,3,2020-08-18,20 days,2.0,21.0,43637.0,NaT,0.0,2020-09-07,18666.0
576,2020-09-24,84,1,2020-08-18,37 days,2.0,21.0,43637.0,NaT,0.0,2020-09-24,18666.0
581,2020-09-27,84,1,2020-08-18,40 days,2.0,21.0,43637.0,NaT,0.0,2020-09-27,18666.0
601,2020-09-29,84,1,2020-08-18,42 days,2.0,21.0,43637.0,NaT,0.0,2020-09-29,18666.0


Next, find the `amount_outstanding` after each made contact using a straightforward *cumulative sum* algorithm:

In [32]:
df['transferred_amount_cumsumed'] = df.groupby(by=['debt_id'])['transferred_amount'].agg(np.cumsum)
df['amount_outstanding'] = df['amount_outstanding'] - df['transferred_amount_cumsumed']
df.drop('transferred_amount_cumsumed', axis=1, inplace=True)

In [33]:
df[df.debt_id == 99][65:75] # Example

,contact_date,debt_id,number_of_contact,enter_date,debt_age,agent_id,collector_id,amount,service_end_date,transferred_amount,date_of_trans,amount_outstanding
416064,2024-03-06,99,1,2020-08-18,1296 days,2.0,17.0,14457.0,NaT,50.0,2024-03-06,12340.0
416648,2024-03-09,99,1,2020-08-18,1299 days,2.0,17.0,14457.0,NaT,0.0,2024-03-09,12340.0
421745,2024-04-04,99,1,2020-08-18,1325 days,2.0,17.0,14457.0,NaT,50.0,2024-04-04,12290.0
426167,2024-05-04,99,1,2020-08-18,1355 days,2.0,17.0,14457.0,NaT,50.0,2024-05-04,12240.0
430674,2024-06-02,99,1,2020-08-18,1384 days,2.0,17.0,14457.0,NaT,50.0,2024-06-02,12190.0
430928,2024-06-04,99,1,2020-08-18,1386 days,2.0,17.0,14457.0,NaT,0.0,2024-06-04,12190.0
433958,2024-07-01,99,1,2020-08-18,1413 days,2.0,17.0,14457.0,NaT,50.0,2024-07-01,12140.0
434260,2024-07-03,99,1,2020-08-18,1415 days,2.0,17.0,14457.0,NaT,0.0,2024-07-03,12140.0
436955,2024-08-01,99,1,2020-08-18,1444 days,2.0,17.0,14457.0,NaT,0.0,2024-08-01,12140.0
437076,2024-08-04,99,1,2020-08-18,1447 days,2.0,17.0,14457.0,NaT,0.0,2024-08-04,12140.0


Let's finalize the data preparation by adding the to service end date column and handling the **nan** values in the dataset *df* and casting the identity variables to categories columns since they do not carry numerical meanings.

Starting by adding `to service end date`, which is the absolute value of `contact date` - `service end date`. But before doing that, we should handel the missing values in the `service end date`:

In [34]:
df.isna().sum()

contact_date              0
debt_id                   0
number_of_contact         0
enter_date                0
debt_age                  0
agent_id                  0
collector_id              0
amount                    7
service_end_date      23033
transferred_amount        6
date_of_trans             1
amount_outstanding       13
dtype: int64

I'll impute the `enter date` for them:

In [35]:
df.loc[df.service_end_date.isna(), 'service_end_date'] = df.loc[df.service_end_date.isna(), 'enter_date']
df['to_service_end_date'] = df['service_end_date'] - df['contact_date']

before droping the reset **nan** values from other columns, let's get a fast look on them:

In [36]:
df[df.transferred_amount.isna() | df.date_of_trans.isna() | df.amount_outstanding.isna() | df.amount.isna()]

,contact_date,debt_id,number_of_contact,enter_date,debt_age,agent_id,collector_id,amount,service_end_date,transferred_amount,date_of_trans,amount_outstanding,to_service_end_date
2568,2020-12-29,86,1,2020-08-18,133 days,2.0,21.0,11781.0000,2010-12-31,NaN,2022-12-29,NaN,-3651 days
920,2020-11-10,94,1,2020-08-18,84 days,2.0,21.0,32220.0000,2020-08-18,NaN,2021-12-29,NaN,-84 days
19859,2021-04-30,95,1,2020-08-18,255 days,2.0,14.0,36206.0000,2020-08-18,NaN,2021-12-29,NaN,-255 days
152471,2022-03-30,43319,2,2021-10-31,150 days,10.0,7.0,253.3550,2021-10-31,NaN,2022-03-30,NaN,-150 days
166245,2022-05-27,203706,1,2021-12-26,152 days,1.0,16.0,87.9100,2021-11-30,NaN,2022-11-24,NaN,-178 days
159568,2022-04-25,238996,1,2022-03-19,37 days,1083.0,19.0,127.4000,2022-03-19,NaN,2022-05-11,NaN,-37 days
168379,2022-06-01,247317,1,2022-06-01,0 days,2089.0,-1.0,NaN,2022-06-01,0.0,2022-06-01,NaN,0 days
175778,2022-06-21,247317,1,2022-06-01,20 days,2089.0,-1.0,NaN,2022-06-01,0.0,2022-06-21,NaN,-20 days
220225,2022-09-25,247317,1,2022-06-01,116 days,2089.0,-1.0,NaN,2022-06-01,0.0,2022-09-25,NaN,-116 days
227496,2022-10-09,247317,1,2022-06-01,130 days,2089.0,-1.0,NaN,2022-06-01,0.0,2022-10-09,NaN,-130 days


As showm, almost important information are missed, so let's remove drop them and let's impute for that row with missing `date of trans` the `date of contact`:

In [37]:
df.loc[df.date_of_trans.isna(), 'date_of_trans'] = df.loc[df.date_of_trans.isna(), 'contact_date']
df.dropna(inplace=True)
df.isna().sum()

contact_date           0
debt_id                0
number_of_contact      0
enter_date             0
debt_age               0
agent_id               0
collector_id           0
amount                 0
service_end_date       0
transferred_amount     0
date_of_trans          0
amount_outstanding     0
to_service_end_date    0
dtype: int64

One last check to duplication rows:

In [38]:
df.duplicated().sum()

0

Finally, let's save the last fotm of data, *df*, in a *.csv* format:

In [39]:
df.to_csv('debt_collection.csv')
df.to_pickle('debt_collection.pkl') # for easier importing

In this notebook we have successfully merged al datasets in one dataframe. The next step is to explore this dataframe.

In [40]:
# debt age, done
# debt status
# good contact (binary | ordinal(1-5)), 
# collection ratio from that sucsess contact, 
# bad debts, 
# to_end_date



# add date_of_trans